In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Firefox
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common import keys
import time
from datetime import date, timedelta, datetime
from urllib.parse import urlsplit
import os
import json
from shutil import copyfile
from collections import Counter
import string
import wptools
import pycountry_convert as pc
import re
import pymorphy2
from pymystem3 import Mystem
mystem = Mystem() 
from sklearn.feature_extraction.text import CountVectorizer
import nltk

def write_json_to_file(d, filename):
    path = 'Desktop//учебка_ВШЭ//Computational_linguistics//Project1//' + filename
    with open(path, 'w', encoding='utf-8') as f:
        f.write(json.dumps(d, ensure_ascii=False, indent=4)) 
        
def get_json_from_file(filename):
    path = 'Desktop//учебка_ВШЭ//Computational_linguistics//Project1//' + filename
    with open(path, 'r', encoding='utf-8') as f:
        d = json.loads(f.read(), object_hook=lambda d: {int(k) if k.lstrip('-').isdigit() else k: v for k, v in d.items()})
    return d

def update_anthems(d):
    copyfile('Desktop//учебка_ВШЭ//Computational_linguistics//Project1//anthems.json', 
             'Desktop//учебка_ВШЭ//Computational_linguistics//Project1//anthems_backup.json')
    with open('Desktop//учебка_ВШЭ//Computational_linguistics//Project1//anthems.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(d, ensure_ascii=False, indent=4)) 
        
def get_lexicon_from_file(filename): 
    path = 'Desktop//учебка_ВШЭ//Computational_linguistics//Project1//' + filename
    lexicon = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            lexicon.append(line.strip('\n').lstrip('\ufeff').lower())   
    return lexicon

In [2]:
anthems = get_json_from_file('anthems.json')

## Задача 1: Частотный анализ 

### Найдем все символы (пунктуацию, цифры), от которых нужно очистить текст

In [6]:
all_anthems = ''
for anthem in anthems.values():
    all_anthems = all_anthems + anthem['text'] + ' '

In [7]:
symbols_to_del = []
for symbol in set(all_anthems):
    if symbol.isalpha() == False:
        symbols_to_del.append(symbol)

In [8]:
symbols_to_del

['3',
 '"',
 '1',
 '-',
 '(',
 ']',
 '»',
 '0',
 ',',
 '2',
 '9',
 '«',
 '4',
 '.',
 '!',
 '5',
 '[',
 ':',
 '/',
 '—',
 ';',
 '6',
 '7',
 '…',
 '„',
 '8',
 ' ',
 '\n',
 "'",
 ')',
 '–',
 '?']

### Предобработка текста: очистка от ненужных символов, лемматизация

In [5]:
def clean(text, symbols_to_del):
    for symbol_to_del in symbols_to_del:
        text = text.replace(symbol_to_del, ' ')
    text = ' '.join([symbol for symbol in text.split() if symbol != ' '])
    return text

def del_stopwords(text, stopwords):
    return ' '.join(word for word in text.split() if word not in stopwords)

In [68]:
# Обрабатываем стоп-слова из НКРЯ

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords = ' '.join(stopwords) 
stopwords_lemmatized = mystem.lemmatize(stopwords)
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend(['наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'индонезия'])

print(stopwords)

['они', 'а', 'же', 'ни', 'потому', 'это', 'время', 'можно', 'человек', 'мочь', 'так', 'и', 'во', 'ну', 'вот', 'здесь', 'уже', 'чтобы', 'более', 'после', 'тот', 'ты', 'через', 'себя', 'я', 'ничто', 'где', 'с', 'не', 'все', 'без', 'когда', 'как', 'нет', 'что', 'быть', 'только', 'теперь', 'тогда', 'он', 'при', 'очень', 'этот', 'который', 'о', 'для', 'за', 'к', 'то', 'даже', 'она', 'бы', 'один', 'от', 'из', 'у', 'надо', 'раз', 'вы', 'чем', 'до', 'со', 'но', 'потом', 'там', 'сказать', 'мы', 'если', 'под', 'ли', 'или', 'на', 'тут', 'еще', 'в', 'кто', 'по', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'индонезия']


In [8]:
anthems_lemmatized = mystem.lemmatize(all_anthems)

In [84]:
anthems_lemmatized_clean = del_stopwords(clean(' '.join(anthems_lemmatized), symbols_to_del), stopwords)

### Сам подсчет частотности 

In [65]:
def get_x_top_ngrams(text, x, n):
    bigram_vectorizer = CountVectorizer(ngram_range=(n, n), min_df=1)
    analyzer = bigram_vectorizer.build_analyzer()
    ngram_to_freq = analyzer(text)
    ngrams_by_freq = dict(Counter(ngram_to_freq).most_common(x))
    return ngrams_by_freq    

In [85]:
get_x_top_ngrams(anthems_lemmatized_clean, 103, 1) # 100 + 3, т.к. несколько со значением 11

{'земля': 88,
 'народ': 73,
 'свобода': 70,
 'родина': 62,
 'страна': 58,
 'слава': 57,
 'сердце': 54,
 'весь': 53,
 'пусть': 50,
 'кровь': 41,
 'жить': 40,
 'бог': 39,
 'давать': 39,
 'мир': 38,
 'любовь': 37,
 'сын': 35,
 'край': 35,
 'свободный': 33,
 'благословлять': 32,
 'небо': 31,
 'да': 31,
 'умирать': 30,
 'честь': 28,
 'пока': 27,
 'отчизна': 26,
 'свет': 25,
 'рука': 25,
 'ребенок': 24,
 'вперед': 23,
 'счастие': 23,
 'великий': 23,
 'флаг': 23,
 'хранить': 23,
 'солнце': 22,
 'гора': 22,
 'море': 22,
 'брат': 22,
 'жизнь': 22,
 'родной': 22,
 'готовый': 22,
 'вставать': 21,
 'сила': 21,
 'мать': 20,
 'отечество': 20,
 'африка': 20,
 'война': 19,
 'любить': 19,
 'предок': 18,
 'день': 18,
 'каждый': 18,
 'душа': 18,
 'славиться': 17,
 'смерть': 17,
 'тысяча': 16,
 'сильный': 16,
 'судьба': 16,
 'враг': 16,
 'имя': 16,
 'единство': 16,
 'верный': 16,
 'надежда': 16,
 'победа': 16,
 'идти': 16,
 'дом': 15,
 'петь': 15,
 'право': 15,
 'единый': 15,
 'вместе': 15,
 'всегда': 15,

In [77]:
get_x_top_ngrams(anthems_lemmatized_clean, 65, 2)

{'готовый умирать': 20,
 'родина родина': 14,
 'объединяться когорта': 10,
 'когорта готовый': 10,
 'умирать готовый': 10,
 'умирать италия': 10,
 'италия позвать': 10,
 'сердце любовь': 9,
 'родной земля': 9,
 'свобода свобода': 8,
 'бог отечество': 8,
 'да здравствовать': 7,
 'родина сердце': 7,
 'господи благословлять': 7,
 'свобода право': 6,
 'позвать объединяться': 6,
 'благословлять африка': 6,
 'оружие гражданин': 6,
 'гражданин формировать': 6,
 'формировать батальон': 6,
 'батальон идти': 6,
 'идти идти': 6,
 'кровь нечистый': 6,
 'тысяча год': 5,
 'решать алжир': 5,
 'алжир жить': 5,
 'жить свидетель': 5,
 'великий народ': 5,
 'боже хранить': 5,
 'вперед вперед': 5,
 'благословлять земля': 5,
 'господь бог': 5,
 'священный страна': 4,
 'сын дочь': 4,
 'народ аргентина': 4,
 'аргентина салют': 4,
 'монарх свобода': 4,
 'солнце свобода': 4,
 'рука вести': 4,
 'вести дорога': 4,
 'дорога слава': 4,
 'пора пока': 4,
 'пока бельгиец': 4,
 'бельгиец фламандец': 4,
 'фламандец валл

## Задача 2: Тематическое моделирование 

In [88]:
for anthem in anthems.values():
    text = anthem['text']
    text_lemmatized = mystem.lemmatize(text)
    anthem['text_lemmatized'] = ''.join(text_lemmatized)
    
update_anthems(anthems)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [9]:
documents = [clean(anthem['text_lemmatized'], symbols_to_del) for anthem in anthems.values()]

In [89]:
stopwords.extend(['африка', 'италия', 'эллин', 'да', 'пока'])

In [207]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=7, stop_words=stopwords)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

tf_vectorizer = CountVectorizer(max_df=1.0, min_df=7, stop_words=stopwords)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [208]:
no_topics = 7
random_seed = 42

In [209]:
nmf = NMF(n_components=no_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)

In [210]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

In [211]:
no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
земля свобода родина пусть слава сердце кровь мир весь честь
Topic 1:
благословлять боже хранить земля народ дорогой счастие давать защищать дочь
Topic 2:
край милый гора труд год день гордый родной солнце святой
Topic 3:
страна народ отечество земля кровь союз всегда год жить родной
Topic 4:
вперед вставать единый бой полный смерть вместе час новый народ
Topic 5:
здравствовать древний гордость гордый честь любовь народ свобода слава флаг
Topic 6:
свободный умирать великий жить народ слава погибать свобода пусть имя

Topic 0:
давать народ счастие отчизна предок страна земля гора сердце весь
Topic 1:
земля бог свобода хранить слава дом народ благословлять небо сын
Topic 2:
родина любовь сердце земля тысяча всегда весь ребенок война небо
Topic 3:
умирать благословлять честь народ весь бог давать свободный земля свобода
Topic 4:
страна жить земля свобода слава пусть кровь народ мир сердце
Topic 5:
свобода весь сердце пусть народ единый слава вперед петь вставать
Topic 6:
край зем

### Возьмем первый набор тем и отредактируем его вручную 

In [196]:
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
земля свобода родина пусть слава сердце кровь мир весь честь
Topic 1:
благословлять боже хранить земля народ дорогой счастие давать защищать дочь
Topic 2:
край милый гора труд год день гордый родной солнце святой
Topic 3:
страна народ отечество земля кровь союз всегда год жить родной
Topic 4:
вперед вставать единый бой полный смерть вместе час новый народ
Topic 5:
здравствовать древний гордость гордый честь любовь народ свобода слава флаг
Topic 6:
свободный умирать великий жить народ слава погибать свобода пусть имя



In [ ]:
# 0 - общепатриотическая
# 1 - бог
# 2 - общепатриотическая (восхищение)
# 3 - общепатриотическая
# 4 - война, свобода
# 5 - общепатриотическая (историческая)
# 6 - смерть за родину

In [194]:
tfidf_feature_names

['берег',
 'битва',
 'благородный',
 'благословлять',
 'бог',
 'боже',
 'бой',
 'борьба',
 'брат',
 'век',
 'великий',
 'вера',
 'верность',
 'верный',
 'вести',
 'весь',
 'ветер',
 'вечный',
 'видеть',
 'вместе',
 'война',
 'волна',
 'вперед',
 'враг',
 'всегда',
 'вставать',
 'герой',
 'глаз',
 'год',
 'гора',
 'гордость',
 'гордый',
 'господь',
 'грудь',
 'давать',
 'день',
 'должный',
 'дом',
 'дорогой',
 'дочь',
 'древний',
 'дух',
 'душа',
 'единство',
 'единый',
 'жизнь',
 'жить',
 'защищать',
 'звезда',
 'здравствовать',
 'земля',
 'знамя',
 'знать',
 'имя',
 'каждый',
 'край',
 'кровь',
 'любить',
 'любовь',
 'мать',
 'милый',
 'мир',
 'море',
 'навсегда',
 'надежда',
 'народ',
 'находить',
 'нация',
 'небо',
 'никогда',
 'новый',
 'огонь',
 'отец',
 'отечество',
 'отчизна',
 'песня',
 'петь',
 'победа',
 'погибать',
 'подымать',
 'полный',
 'последний',
 'право',
 'предок',
 'прекрасный',
 'приходить',
 'пусть',
 'путь',
 'радость',
 'ребенок',
 'река',
 'родина',
 'родной',


In [229]:
topics_dict = {
    'общепатриотическая':'земля слава сердце жить честь труд родной солнце верный великий дом гордый гордость здравствовать сын мать отец ребенок любить любовь нация флаг',
    'война':'вперед вставать бой битва смерть война мир час свобода кровь борьба герой враг единство единый свободный защищать знамя умирать погибать победа проливать', 
    'бог':'бог боже благословлять вера господь дух душа святой священный хранить защищать',
    'описание': 'край милый гора солнце волна море дорогой скала река сиять цвести яркий свет прекрасный'
}

In [3]:
vectorizer = TfidfVectorizer(stop_words=stopwords)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

#def jaccard_sim(text1, text2): 
#    a = set(text1.split()) 
#    b = set(text2.split())
#    c = a.intersection(b)
#    return float(len(c)) / (len(a) + len(b) - len(c))

In [231]:
for anthem in anthems.values():
    text = anthem['text_lemmatized']
    topic_to_similarity = {}
    for topic_name, topic_features in topics_dict.items():
        similarity = cosine_sim(text, topic_features)
        topic_to_similarity[topic_name] = similarity
    anthem['topic_to_similarity'] = topic_to_similarity
    
update_anthems(anthems)

## Задача 3: Синтаксический анализ (выявление "сюжетов", родина - она какая?) 

In [6]:
# update udpipe-lemmas with mystem-lemmas

n = 0
for anthem in anthems.values():
    print('Anthem', n, 'out of ~93')
    for ud_graph in anthem['ud_graphs']:
        n_tokens = len(ud_graph)
        for i in range(0, n_tokens):
            lemma = ud_graph[i]['lemma']
            if lemma != None and lemma.isalpha():
                if type(lemma) == list:
                    lemma = lemma[0].lower()
                else:
                    lemma = lemma.lower()
                ud_graph[i]['lemma'] = mystem.lemmatize(lemma)
    n += 1
    
update_anthems(anthems)

Anthem 0 out of ~93
Anthem 1 out of ~93
Anthem 2 out of ~93
Anthem 3 out of ~93
Anthem 4 out of ~93
Anthem 5 out of ~93
Anthem 6 out of ~93
Anthem 7 out of ~93
Anthem 8 out of ~93
Anthem 9 out of ~93
Anthem 10 out of ~93
Anthem 11 out of ~93
Anthem 12 out of ~93
Anthem 13 out of ~93
Anthem 14 out of ~93
Anthem 15 out of ~93
Anthem 16 out of ~93
Anthem 17 out of ~93
Anthem 18 out of ~93
Anthem 19 out of ~93
Anthem 20 out of ~93
Anthem 21 out of ~93
Anthem 22 out of ~93
Anthem 23 out of ~93
Anthem 24 out of ~93
Anthem 25 out of ~93
Anthem 26 out of ~93
Anthem 27 out of ~93
Anthem 28 out of ~93
Anthem 29 out of ~93
Anthem 30 out of ~93
Anthem 31 out of ~93
Anthem 32 out of ~93
Anthem 33 out of ~93
Anthem 34 out of ~93
Anthem 35 out of ~93
Anthem 36 out of ~93
Anthem 37 out of ~93
Anthem 38 out of ~93
Anthem 39 out of ~93
Anthem 40 out of ~93
Anthem 41 out of ~93
Anthem 42 out of ~93
Anthem 43 out of ~93
Anthem 44 out of ~93
Anthem 45 out of ~93
Anthem 46 out of ~93
Anthem 47 out of ~93
An

In [24]:
# save lemmas in strings, not in lists
# fix mystem mistakes

for anthem in anthems.values():
    for sentence in anthem['ud_graphs']:
        for token in sentence:
            if type(token['lemma']) == list:
                token['lemma'] = token['lemma'][0]
            if token['lemma'] == 'тонаш':
                token['lemma'] = 'наш'
            if token['lemma'] == 'милая':
                token['lemma'] = 'милый'
                

update_anthems(anthems)

In [10]:
anthems[89]

{'name': 'Эфиопия',
 'link': 'http://discoveric.ru/anthem/efiopiya',
 'text': 'Гражданство пользуется большим уважением в нашей Эфиопии;\nГордость за нацию видна, сияя от края до края.\nЗа мир, за справедливость, за свободу народа,\nВ равенстве и в любви мы едины.\nНа прочном фундаменте мы не отрекаемся от человечности;\nМы народ, что живет трудом.\nЧудесное место традиций, хозяйка гордого наследия,\nМать природной доблести, мать отважного народа.\nМы защитим тебя — это наш долг;\nНаша Эфиопия, живи! И давайте гордиться тобой!',
 'name_en': 'Ethiopia',
 'state_characteristics': ['federalism',
  'dominant-party system',
  'parliamentary system',
  'constitutional republic'],
 'religion': {},
 'area': 1104300,
 'former_colony': 1,
 'population': 102403196,
 'continent': 'AF',
 'secular': 1,
 'text_lemmatized': 'гражданство пользоваться большой уважение в наш эфиопия;\nгордость за нация видный, сиять от край до край.\nза мир, за справедливость, за свобода народ,\nв равенство и в любовь мы

In [29]:
# Родина -- она какая?
# ctag - ADJ/DET

fatherland_characteristic = []

for anthem in anthems.values():
    fatherland_synonyms = ['страна', 'родина', 'отчизна', 'земля', 'край', 'отечество', 'государство', 
                           'королевство', 'монархия', 'республика', 'остров', 'рим', 'африка', 
                           'эмират', 'княжество', 'мать', 'корея'] # + country name
    fatherland_synonyms.append(anthem['name'].lower())
    for sentence in anthem['ud_graphs']:
        
        fatherland_lemmas = [] # list for saving addresses of 'fatherland' words
        
        # get all 'fatherland' words in sentence
        for token in sentence:
            if token['lemma'] in fatherland_synonyms:
                fatherland_lemmas.append(token['address'])
        
        # get all words describing 'fatherland' words in sentence -- either ADJ or DET
        for token in sentence:
            if token['head'] in fatherland_lemmas and token['ctag'] in ['ADJ', 'DET']:
                fatherland_characteristic.append(token['lemma'])
                
print(fatherland_characteristic)

['священный', 'весь', 'пашня', 'великий', 'испытанный', 'испытанный', 'свой', 'светлый', 'заветный', 'единый', 'великий', 'свободный', 'независимый', 'дорогой', 'священный', 'благородный', 'весь', 'милый', 'униженный', 'любимый', 'этот', 'благородный', 'любимый', 'блаженный', 'благородный', 'блаженный', 'благородный', 'они', 'отстоять', 'родной', 'вечный', 'великий', 'немецкий', 'немецкий', 'родной', 'мой', 'прекрасный', 'старый', 'старый', 'богатый', 'свободный', 'мой', 'гордый', 'свободный', 'собственный', 'он', 'наш', 'прекрасный', 'родной', 'мой', 'свой', 'мой', 'объединять', 'мой', 'мой', 'великий', 'великий', 'независимый', 'мой', 'мой', 'великий', 'независимый', 'великий', 'замечательный', 'богатый', 'священный', 'победоносный', 'лучезарный', 'этот', 'весь', 'она', 'наш', 'наш', 'рабыня', 'родной', 'голый', 'мой', 'мой', 'мой', 'мой', 'родной', 'мой', 'широкий', 'необъятный', 'сплоченный', 'независимый', 'такой', 'мой', 'мой', 'мой', 'родной', 'мой', 'древний', 'гордый', 'древни

In [30]:
len(fatherland_characteristic)

194

In [31]:
fatherland_characteristic_by_freq = dict(Counter(fatherland_characteristic).most_common())

In [32]:
fatherland_characteristic_by_freq

{'мой': 27,
 'наш': 19,
 'родной': 16,
 'великий': 7,
 'весь': 5,
 'дорогой': 5,
 'прекрасный': 5,
 'свой': 4,
 'независимый': 4,
 'благородный': 4,
 'милый': 4,
 'священный': 3,
 'свободный': 3,
 'любимый': 3,
 'этот': 3,
 'древний': 3,
 'родимый': 3,
 'испытанный': 2,
 'блаженный': 2,
 'вечный': 2,
 'немецкий': 2,
 'старый': 2,
 'богатый': 2,
 'гордый': 2,
 'обетованный': 2,
 'о': 2,
 'самый': 2,
 'храбрый': 2,
 'мальдивский': 2,
 'любить': 2,
 'чешский': 2,
 'пашня': 1,
 'светлый': 1,
 'заветный': 1,
 'единый': 1,
 'униженный': 1,
 'они': 1,
 'отстоять': 1,
 'собственный': 1,
 'он': 1,
 'объединять': 1,
 'замечательный': 1,
 'победоносный': 1,
 'лучезарный': 1,
 'она': 1,
 'рабыня': 1,
 'голый': 1,
 'широкий': 1,
 'необъятный': 1,
 'сплоченный': 1,
 'такой': 1,
 'кхмерский': 1,
 'уважаемый': 1,
 'должный': 1,
 'последний': 1,
 'тот': 1,
 'плодородный': 1,
 'бесплатный': 1,
 'уверенный': 1,
 'аннсвящий': 1,
 'твой': 1,
 'многообразнать': 1,
 'возлюбленная': 1,
 'новый': 1,
 'покой': 

In [34]:
# Родина -- земля чего?

fatherland_synonyms = ['страна', 'земля', 'край', 'государство', 'королевство', 
                       'республика', 'остров', 'княжество', 'мать'] 
fatherland_characteristic = []

for anthem in anthems.values():
    for sentence in anthem['ud_graphs']:
        fatherland_lemmas = []
        for token in sentence:
            
            if token['lemma'] in fatherland_synonyms:
                fatherland_lemmas.append(token['address'])
                
            if token['head'] in fatherland_lemmas and token['ctag'] == 'NOUN' and 'Case=Gen' in token['feats']:
                fatherland_characteristic.append(token['lemma'])
                
print(fatherland_characteristic)

['земля', 'гора', 'поток', 'собор', 'молотов', 'сын', 'свидетель', 'милый', 'отцов', 'бельгиец', 'красота', 'так', 'подарок', 'отцов', 'отцов', 'отцов', 'предок', 'море', 'волна', 'сирота', 'вдова', 'слава', 'надежда', 'гостеприимство', 'герой', 'вред', 'предок', 'процветание', 'саванна', 'круча', 'круча', 'свобода', 'свобода', 'свобода', 'гулять', 'мужество', 'солнце', 'слово', 'отцов', 'слава', 'радость', 'доблесть', 'народ']


In [35]:
dict(Counter(fatherland_characteristic).most_common())

{'отцов': 5,
 'свобода': 3,
 'предок': 2,
 'слава': 2,
 'круча': 2,
 'земля': 1,
 'гора': 1,
 'поток': 1,
 'собор': 1,
 'молотов': 1,
 'сын': 1,
 'свидетель': 1,
 'милый': 1,
 'бельгиец': 1,
 'красота': 1,
 'так': 1,
 'подарок': 1,
 'море': 1,
 'волна': 1,
 'сирота': 1,
 'вдова': 1,
 'надежда': 1,
 'гостеприимство': 1,
 'герой': 1,
 'вред': 1,
 'процветание': 1,
 'саванна': 1,
 'гулять': 1,
 'мужество': 1,
 'солнце': 1,
 'слово': 1,
 'радость': 1,
 'доблесть': 1,
 'народ': 1}

## *Анализ тем 

In [106]:
def get_topics_rate_under_condition(key, value):
    topics = ['описание', 'бог', 'война', 'общепатриотическая']
    topics_points = {topic: 0 for topic in topics}
    rate_to_point = {0:4, 1:3, 2:2, 3:1}
    
    n_countries = 0
    for anthem in anthems.values():
        if anthem[key] == value:
            n_countries += 1
            topics_sorted = anthem['topics_sorted']
            for topic in topics:
                if topic in topics_sorted:
                    topics_points[topic] += rate_to_point[topics_sorted.index(topic)]
    topics_points = {topic: value/(n_countries*4) for topic, value in topics_points.items()}
    print(sorted(topics_points.items(), reverse=True, key=lambda x: x[1]))
    #print(n_countries)